In [7]:
from neuron import n, gui
from neuron.units import ms, mV
n.load_file("stdrun.hoc")

1.0

In [2]:
class Cell:
    def __init__(self, gid, x, y, z, theta):
        self._gid = gid
        self._setup_morphology()
        self.all = self.soma.wholetree()
        self._setup_biophysics()
        self.x = self.y = self.z = 0  # <-- NEW
        n.define_shape()
        self._rotate_z(theta)  # <-- NEW
        self._set_position(x, y, z)  # <-- NEW

    def __repr__(self):
        return "{}[{}]".format(self.name, self._gid)

    # everything below here is NEW

    def _set_position(self, x, y, z):
        for sec in self.all:
            for i in range(sec.n3d()):
                sec.pt3dchange(
                    i,
                    x - self.x + sec.x3d(i),
                    y - self.y + sec.y3d(i),
                    z - self.z + sec.z3d(i),
                    sec.diam3d(i),
                )
        self.x, self.y, self.z = x, y, z

    def _rotate_z(self, theta):
        """Rotate the cell about the Z axis."""
        for sec in self.all:
            for i in range(sec.n3d()):
                x = sec.x3d(i)
                y = sec.y3d(i)
                c = n.cos(theta)
                s = n.sin(theta)
                xprime = x * c - y * s
                yprime = x * s + y * c
                sec.pt3dchange(i, xprime, yprime, sec.z3d(i), sec.diam3d(i))

In [3]:
## Run only for jupyter
class BallAndStick(Cell):
    name = "BallAndStick"

    def _setup_morphology(self):
        self.soma = n.Section("soma", self)
        self.dend = n.Section("dend", self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100  # Axial resistance in Ohm * cm
            sec.cm = 1  # Membrane capacitance in micro Farads / cm^2
        self.soma.insert(n.hh)
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003  # Leak conductance in S/cm2
            seg.hh.el = -54.3  # Reversal potential in mV
        # Insert passive current in the dendrite
        self.dend.insert(n.pas)
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -65  # Leak reversal potential mV

In [4]:
def create_n_BallAndStick(num, r):
    """num = number of cells; r = radius of circle"""
    cells = []
    for i in range(num):
        theta = i * 2 * n.PI / num
        cells.append(BallAndStick(i, n.cos(theta) * r, n.sin(theta) * r, 0, theta))
    return cells

In [5]:
my_cells = create_n_BallAndStick(7, 50)

In [8]:
ps = n.PlotShape(True)
ps.show(0)

1

In [9]:
n.topology()


|-|       BallAndStick[0].soma(0-1)
   `|       BallAndStick[0].dend(0-1)
|-|       BallAndStick[1].soma(0-1)
   `|       BallAndStick[1].dend(0-1)
|-|       BallAndStick[2].soma(0-1)
   `|       BallAndStick[2].dend(0-1)
|-|       BallAndStick[3].soma(0-1)
   `|       BallAndStick[3].dend(0-1)
|-|       BallAndStick[4].soma(0-1)
   `|       BallAndStick[4].dend(0-1)
|-|       BallAndStick[5].soma(0-1)
   `|       BallAndStick[5].dend(0-1)
|-|       BallAndStick[6].soma(0-1)
   `|       BallAndStick[6].dend(0-1)



1.0

In [9]:
?n

Call signature: n(*args, **kwargs)
Type:           _NEURON_INTERFACE
String form:    <TopLevelNEURONInterface>
File:           ~/anaconda3/envs/Neuron/lib/python3.10/site-packages/neuron/__init__.py
Docstring:     
neuron.n

neuron.n is the top-level NEURON inteface, starting in NEURON 9.

>>> from neuron import n
>>> n
<TopLevelNEURONInterface>

Most NEURON classes and functions are defined in the n namespace
and can be accessed as follows:

>>> v = n.Vector(10)
>>> soma = n.Section("soma")
>>> input = n.IClamp(soma(0.5))
>>> n.finitialize(-65)

Each built-in class has its own type, so for the above definitions we have:

>>> type(v)
<class 'hoc.Vector'>
>>> type(soma)
<class 'nrn.Section'>

But since ``IClamp`` is defined by a MOD file:

>>> type(input)
<class 'hoc.HocObject'>

Other submodules of neuron exist, including rxd and units.

You can see the functions, classes, etc available inside n via dir(n)
and can get help on each via the standard Python help system, e.g.,

>>> help(n.